In [1]:
import pandas as pd
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
path_deg = '/home/juan/Desktop/juan/bio/mirna_mite/data/res/res_deg.csv'
path_mir = '/home/juan/Desktop/juan/bio/mirna_mite/data/res/all_results.csv'


In [ ]:
df_mir = pd.read_csv(path_mir, sep='\t')
df_mir.rename(columns={'MITE':'MITE_prod'}, inplace=True)
total_mir = len(df_mir.index)
print(total_mir)
df_mir.head(2)

In [ ]:
df_deg = pd.read_csv(path_deg, sep='\t')
df_deg.rename(columns={'MITE':'MITE_target'}, inplace=True)
print(len(df_deg.index))
df_deg.head(2)

In [ ]:
has_target = 0
df_mir['has_target'] == 0:
has_same_target = 0
for k,v in df_mir.iterrows():
    df_target = df_deg[df_deg.mir == v.mir]
    if len(df_target.index) > 0:
            has_target += 1
        prods = v.MITE_prod.split(',')
        target = v.MITE_target.split(',')
        prods = set(filter(None, prods))
        target = set(filter(None, target))
        inter = target.intersection(prods)
        if df_mir.at[k, 'has_target'] == 0:
            df_mir.at[k, 'has_target'] = 1 
        if inter:
            has_same_target += 1
            print(v.SiteID, v.Name, inter)
            df_mir.at[k, 'has_target'] = 2

In [ ]:
has_target * 100 / total_mir

In [ ]:
has_same_target * 100 / total_mir

In [ ]:
df_mir['target_intron'] = 0
df_mir['target_exon'] = 0
df_mir['target_three_prime'] = 0
df_mir['target_five_prime'] = 0

In [ ]:
#GENE TARGET REGION